# **Argubot**: Un Chatbot sobre la tesis doctoral de Tomás Manzur "Construcción de argumentos y controversias socio-técnicas: Análisis de la conflictividad surgida en la discusión del Plan Provincial de Ordenamiento Territorial de la Provincia de Mendoza"


## Introducción
El **ArguBot** es un chatbot revolucionario, implementado en un **Jupyter Notebook**, diseñado para ofrecer asistencia especializada en consultas sobre la tesis doctoral de Tomás Manzur, "Construcción de Argumentos y Controversias Socio-Técnicas". Este bot se destaca por su análisis avanzado y su enfoque en la **privacidad del usuario** y el uso de **fuentes de código abierto**.

**Corre el siguiente bloque de código para instalar e importar las librerías necesarias**


In [ ]:
!pip install -q llama-index chromadb
import os
import chromadb
import requests
from getpass import getpass
from llama_index.llms import OpenAI, Gemini
from llama_index import SimpleDirectoryReader, VectorStoreIndex
from llama_index.response.notebook_utils import display_response
from llama_index.vector_stores import ChromaVectorStore
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.storage.storage_context import StorageContext
from llama_index.service_context import ServiceContext
from llama_index.prompts import PromptTemplate

## Ingreso de Google API Key

Para comenzar a interactuar con el **ArguBot**, es necesario que ingreses tu **Google API Key**. Por favor, introduce tu clave en el campo de entrada a continuación.

Recuerda que debes tener habilitada la Generative Language API en GCP para utilizar el bot. Tienes una guía detallada para realizarlo en el *Readme* del proyecto.

### Privacidad del Usuario
La interacción con el bot requiere una **API_KEY**, ingresada de manera segura mediante `getpass`. Con `getpass`, tu clave **no se almacena** y se mantiene confidencial dentro de la sesión actual del usuario. Esto garantiza la **confidencialidad** y la **protección de los datos del usuario**. Tu privacidad y seguridad son nuestra máxima prioridad.

**Por favor, corre el siguiente bloque de código e ingresa tu Google API Key:**


In [ ]:
GOOGLE_API_KEY = getpass('Enter your Gogle secret API Key: ')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


Enter your Gogle secret API Key: ··········


# Bienvenido a ArguBot

Bienvenido a ArguBot, el chatbot interactivo diseñado para ayudarte a explorar los ricos y complejos temas cubiertos en la tesis doctoral de Tomás Manzur sobre "Construcción de Argumentos y Controversias Socio-Técnicas", enfocándose en el Plan Provincial de Ordenamiento Territorial de Mendoza.

Como usuario, tienes la oportunidad de adentrarte en varios aspectos de esta tesis, incluyendo las complejidades de la planificación territorial, la dinámica de las controversias socio-técnicas y un análisis detallado de argumentación. Ya seas estudiante, investigador o simplemente alguien interesado en estos temas, ArguBot está aquí para guiarte a través de los contenidos y hallazgos de la tesis.

**Corre el siguiente bloque de código para configurar el modelo**

*Esto puede tardar varios minutos*

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

with requests.get('https://drive.google.com/file/d/1iFHzW_QO_f6LJzBiOB5ZM7taxAvf7Jgr/view?usp=drive_link', stream=True) as r:
    r.raise_for_status()
    with open('/content/src/data.txt', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

documents = SimpleDirectoryReader('src').load_data()

llm = Gemini(temperature=0.2, model="gemini-pro")

db = chromadb.PersistentClient(path="./chroma_db_HF")
chroma_collection = db.get_or_create_collection("Tesis_Tomas_Manzur")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

svc = ServiceContext.from_defaults(embed_model=embed_model,llm=llm)
stc = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=stc, service_context=svc
)
query_engine = index.as_query_engine()

template = (
    "Dado el contexto que te proporcionare responde las preguntas \n"
    "Contexto:\n"
    "################################\n"
    "{context_str}"
    "################################\n"
    "Responde en español como si fueras sociologo experto en controversias sociotécnicas y en análisis de conflictos sociales, políticos, ambientales y territoriales: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)


## Interactúa con ArguBot

Aquí hay algunos tipos de preguntas o interacciones que puedes tener con ArguBot:

1. **Consultas de Contenido**: Pide información específica de la tesis.
   - Ejemplo: "¿Cuál es el argumento principal sobre la gestión de recursos hídricos en la tesis?"

2. **Análisis de Argumentos**: Profundiza en el análisis de los argumentos presentados en la tesis.
   - Ejemplo: "¿Puedes desglosar los argumentos a favor y en contra del desarrollo urbano en Mendoza?"

3. **Contextualización de Conflictos**: Explora los conflictos socio-técnicos dentro de la planificación territorial de Mendoza.
   - Ejemplo: "¿Cuáles son los principales conflictos en el Plan Provincial de Ordenamiento Territorial y cómo impactan a las comunidades locales?"

4. **Perspectivas Metodológicas**: Pregunta sobre las metodologías de investigación utilizadas en la tesis.
   - Ejemplo: "¿Cómo contribuyó el enfoque de investigación cualitativa a los hallazgos de la tesis?"

5. **Discusión sobre Hallazgos**: Discute hallazgos específicos de la tesis.
   - Ejemplo: "¿Qué revela la tesis sobre el papel de los actores sociales en la planificación territorial de Mendoza?"

Siéntete libre de hacer cualquier pregunta dentro de estos dominios o áreas relacionadas, y haré mi mejor esfuerzo para proporcionar información perspicaz y precisa basada en la tesis. ¡Exploramos juntos el fascinante mundo del análisis de argumentos socio-técnicos!

**Corre el siguiente bloque de código para iniciar el chat con ArguBot**

In [ ]:
i = 0
while i < 10:
  inp = input("User: ")
  response = query_engine.query(inp)
  display_response(response)
  i + 1